<div style="  background: linear-gradient(145deg, #0f172a, #1e293b);  border: 4px solid transparent;  border-radius: 14px;  padding: 18px 22px;  margin: 12px 0;  font-size: 26px;  font-weight: 600;  color: #f8fafc;  box-shadow: 0 6px 14px rgba(0,0,0,0.25);  background-clip: padding-box;  position: relative;">  <div style="    position: absolute;    inset: 0;    padding: 4px;    border-radius: 14px;    background: linear-gradient(90deg, #06b6d4, #3b82f6, #8b5cf6);    -webkit-mask:       linear-gradient(#fff 0 0) content-box,       linear-gradient(#fff 0 0);    -webkit-mask-composite: xor;    mask-composite: exclude;    pointer-events: none;  "></div>    <b>Two-Output Models & Advanced Keras Architectures</b>    <br/>  <span style="color:#9ca3af; font-size: 18px; font-weight: 400;">(Classification, Regression, and Skip Connections)</span></div>

## Table of Contents

1. [Simple Model with 2 Outputs](#section-1)
2. [Fitting and Inspecting a 2-Output Model](#section-2)
3. [Evaluating a 2-Output Model](#section-3)
4. [Single Model for Classification and Regression](#section-4)
5. [Compiling a Multi-Output Model](#section-5)
6. [Fitting the Combination Model](#section-6)
7. [Inspecting Weights and Manual Verification](#section-7)
8. [Evaluating the Combination Model](#section-8)
9. [Advanced Concepts: Wrap-up & Skip Connections](#section-9)
10. [Conclusion](#section-10)

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 1. Simple Model with 2 Outputs</span><br>

In this section, we explore how to create a neural network using the Keras Functional API that takes a single input but produces two distinct outputs. This is useful when you want to predict two related variables simultaneously (e.g., the score of Team 1 and the score of Team 2).

### Architecture
The model consists of:
1.  **Input Layer**: Accepts a tensor of shape `(1,)`.
2.  **Output Layer**: A Dense layer with `2` units. This single layer produces two values.

### Code Implementation



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np

# 1. Define the Input
input_tensor = Input(shape=(1,))

# 2. Define the Output
# We use a Dense layer with 2 units to predict 2 values (e.g., score_1 and score_2)
output_tensor = Dense(2)(input_tensor)

# 3. Build the Model
model = Model(input_tensor, output_tensor)

# 4. Compile the Model
# We use 'mean_absolute_error' as the loss function for regression
model.compile(optimizer='adam', loss='mean_absolute_error')

print("Model compiled successfully.")
model.summary()



<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> Even though there are two outputs, because they come from a single Dense layer, Keras treats this as a single tensor output of shape (batch_size, 2).</div>

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 2. Fitting and Inspecting a 2-Output Model</span><br>

To fit this model, our target variable `y` must match the shape of the output layer. Since the output layer has 2 units, `y` must have 2 columns.

### Data Preview
The dataset used represents tournament games. We use `seed_diff` as the input to predict both `score_1` and `score_2`.

| | seed_diff | score_1 | score_2 |
|---|---|---|---|
| 0 | -3 | 41 | 50 |
| 1 | 4 | 61 | 55 |
| 2 | 5 | 59 | 63 |
| 3 | 3 | 50 | 41 |
| 4 | 1 | 54 | 63 |

### Implementation: Fitting the Model



In [ ]:
# Creating the sample data from the document
data = {
    'seed_diff': [-3, 4, 5, 3, 1],
    'score_1': [41, 61, 59, 50, 54],
    'score_2': [50, 55, 63, 41, 63]
}
games_tourney_train = pd.DataFrame(data)

# Display the head
print(games_tourney_train.head())

# Prepare X and y
X = games_tourney_train[['seed_diff']]
y = games_tourney_train[['score_1', 'score_2']]

# Fit the model
# Note: We use a small number of epochs here for demonstration
history = model.fit(X, y, epochs=100, verbose=0)
print("Training complete.")



### Inspecting Model Weights
After training, we can inspect the weights. Since we have 1 input and 2 outputs, the weight matrix will be shape `(1, 2)` and the bias vector will be shape `(2,)`.



In [ ]:
# Get weights
weights = model.get_weights()

print("Weights (Input -> Output):")
print(weights[0])
print("\nBiases:")
print(weights[1])



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 3. Evaluating a 2-Output Model</span><br>

Evaluation works similarly to fitting. We provide the test data `X` and the 2-column target `y`. The model returns a single scalar loss value representing the Mean Absolute Error averaged over both outputs.



In [ ]:
# Creating synthetic test data for evaluation
test_data = {
    'seed_diff': [2, -1, 0],
    'score_1': [55, 45, 50],
    'score_2': [48, 52, 50]
}
games_tourney_test = pd.DataFrame(test_data)

X_test = games_tourney_test[['seed_diff']]
y_test = games_tourney_test[['score_1', 'score_2']]

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"\nMean Absolute Error on Test Data: {loss}")



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 4. Single Model for Classification and Regression</span><br>

We can build a model that performs **Regression** (predicting a number) and **Classification** (predicting a binary outcome) simultaneously.

### Architecture Design
1.  **Input**: `seed_diff`
2.  **Regression Output**: Predicts the score difference (linear activation).
3.  **Classification Output**: Predicts if Team 1 won (sigmoid activation).
4.  **Connection**: In this specific example, the Classification output layer takes the **Regression Output** as its input. This creates a dependency chain.



In [ ]:
from tensorflow.keras.layers import Input, Dense

# 1. Input Layer
input_tensor = Input(shape=(1,))

# 2. Regression Output
# Predicts a continuous value (e.g., score difference)
output_tensor_reg = Dense(1, name='regression_output')(input_tensor)

# 3. Classification Output
# Takes the regression output as input!
# Uses sigmoid to predict probability (0 to 1)
output_tensor_class = Dense(1, activation='sigmoid', name='classification_output')(output_tensor_reg)



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 5. Compiling a Multi-Output Model</span><br>

When a model has multiple output layers, we define the model by passing a **list** of outputs. We also compile it using a **list** of loss functions corresponding to each output.



In [ ]:
from tensorflow.keras.models import Model

# Define the model with 1 input and 2 distinct output tensors
model_combo = Model(input_tensor, [output_tensor_reg, output_tensor_class])

# Compile the model
# Loss 1 (Regression): Mean Absolute Error
# Loss 2 (Classification): Binary Crossentropy
model_combo.compile(
    loss=['mean_absolute_error', 'binary_crossentropy'],
    optimizer='adam'
)

model_combo.summary()



<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> The order of losses in the list must match the order of outputs defined in the `Model(inputs, [outputs])` list.</div>

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 6. Fitting the Combination Model</span><br>

To fit the model, we pass the input `X` and a **list** of targets `[y_reg, y_class]`.



In [ ]:
# Preparing data for the combined model
# We need 'score_diff' (regression target) and 'won' (classification target)
games_tourney_train['score_diff'] = games_tourney_train['score_1'] - games_tourney_train['score_2']
games_tourney_train['won'] = (games_tourney_train['score_diff'] > 0).astype(int)

X = games_tourney_train[['seed_diff']]
y_reg = games_tourney_train[['score_diff']]
y_class = games_tourney_train[['won']]

# Fit the model
# Notice y is a list: [regression_target, classification_target]
history_combo = model_combo.fit(
    X, 
    [y_reg, y_class], 
    epochs=100, 
    verbose=0
)

print("Combined model training complete.")



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 7. Inspecting Weights and Manual Verification</span><br>

We can inspect the weights to understand how the regression output feeds into the classification output.

### Extracting Weights
The model has two dense layers.
1.  Layer 1: Input -> Regression Output
2.  Layer 2: Regression Output -> Classification Output



In [ ]:
weights = model_combo.get_weights()

print("Layer 1 Weights (Input -> Reg):", weights[0])
print("Layer 1 Bias:", weights[1])
print("Layer 2 Weights (Reg -> Class):", weights[2])
print("Layer 2 Bias:", weights[3])



### Manual Calculation Verification
We can verify the model's prediction logic using the sigmoid function from `scipy`.

$$ \text{Sigmoid}(x) = \frac{1}{1 + e^{-x}} $$

The classification output is calculated as:
$$ \text{Class Output} = \text{Sigmoid}(\text{Reg Output} \times W_2 + b_2) $$



In [ ]:
from scipy.special import expit as sigmoid

# Example values extracted from a hypothetical training run (or use current weights)
# Let's use the actual trained weights from the code above
w1 = weights[0][0][0]
b1 = weights[1][0]
w2 = weights[2][0][0]
b2 = weights[3][0]

# Assume an input value of 1.0
input_val = 1.0

# Calculate Regression Output
reg_out = input_val * w1 + b1

# Calculate Classification Output manually
manual_class_out = sigmoid(reg_out * w2 + b2)

print(f"Input: {input_val}")
print(f"Regression Output (Manual): {reg_out}")
print(f"Classification Output (Manual): {manual_class_out}")

# Verify with Keras prediction
keras_pred = model_combo.predict([input_val], verbose=0)
print(f"Keras Class Prediction: {keras_pred[1][0][0]}")



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 8. Evaluating the Combination Model</span><br>

When evaluating a multi-output model, Keras returns a list of metrics:
1.  **Total Loss**: Sum of all losses.
2.  **Loss 1**: Loss for the first output (Regression).
3.  **Loss 2**: Loss for the second output (Classification).



In [ ]:
# Prepare test data
games_tourney_test['score_diff'] = games_tourney_test['score_1'] - games_tourney_test['score_2']
games_tourney_test['won'] = (games_tourney_test['score_diff'] > 0).astype(int)

X_test = games_tourney_test[['seed_diff']]
y_reg_test = games_tourney_test[['score_diff']]
y_class_test = games_tourney_test[['won']]

# Evaluate
evaluation = model_combo.evaluate(X_test, [y_reg_test, y_class_test])

print("\nEvaluation Results:")
print(f"Total Loss: {evaluation[0]}")
print(f"Regression Loss (MAE): {evaluation[1]}")
print(f"Classification Loss (Binary Crossentropy): {evaluation[2]}")



***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 9. Advanced Concepts: Wrap-up & Skip Connections</span><br>

### Course Summary
Throughout this advanced Keras exploration, we have covered:
*   **Functional API**: Building flexible graphs of layers.
*   **Shared Layers**: Using the same layer instance multiple times (Siamese networks).
*   **Categorical Embeddings**: Handling high-cardinality categorical data.
*   **Multiple Inputs**: Merging text, images, and numerical data.
*   **Multiple Outputs**: Predicting regression and classification targets simultaneously.

### Skip Connections (Residual Connections)
Skip connections are a powerful technique used in deep networks (like ResNet) to solve the vanishing gradient problem. They allow the model to "skip" layers and pass information directly deeper into the network.

#### Implementation of Skip Connections
In the example below, the `input_tensor` is concatenated with the `hidden_tensor` before the final output. This preserves the original input features alongside the learned features.



In [ ]:
from tensorflow.keras.layers import Concatenate

# 1. Input
input_tensor = Input((100,))

# 2. Deep Hidden Layers
hidden_tensor = Dense(256, activation='relu')(input_tensor)
hidden_tensor = Dense(256, activation='relu')(hidden_tensor)
hidden_tensor = Dense(256, activation='relu')(hidden_tensor)

# 3. Skip Connection
# Concatenate the original input with the processed hidden output
output_tensor = Concatenate()([input_tensor, hidden_tensor])

# 4. Final Output
output_tensor = Dense(256, activation='relu')(output_tensor)

# Build Model
model_skip = Model(input_tensor, output_tensor)
model_skip.summary()



<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> Skip connections smooth the loss landscape, making it easier for optimization algorithms (like Adam) to find the global minimum.</div>

***

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 10. Conclusion</span><br>

In this notebook, we successfully transitioned from simple single-output models to complex multi-output architectures using the Keras Functional API.

**Key Takeaways:**
1.  **Flexibility**: The Functional API allows for non-linear topologies, shared layers, and multiple inputs/outputs.
2.  **Multi-Task Learning**: We can train a single model to perform both regression and classification, potentially improving performance by learning shared representations.
3.  **Evaluation**: Multi-output models return a list of losses, allowing granular analysis of model performance.
4.  **Architecture Patterns**: Advanced patterns like Skip Connections are easily implemented using `Concatenate` or `Add` layers.

**Next Steps:**
*   Experiment with **Shared Layers** for comparing inputs (e.g., document similarity).
*   Apply **Embeddings** to categorical inputs in multi-input models.
*   Try implementing a **ResNet** block using the skip connection pattern demonstrated above.
